## Spark SQL and DataFrames: Interacting with External Data Sources

### In this chapter, we will focus on how Spark SQL interfaces with externalcomponents. Specifically, we discuss how Spark SQL allows you to:
• Use user-defined functions for both Apache Hive and Apache Spark.<BR>
• Connect with external data sources such as JDBC and SQL databases, Post‐
greSQL, MySQL, Tableau, Azure Cosmos DB, and MS SQL Server.<BR>
• Work with simple and complex types, higher-order functions, and common relational operators

### User Defined Functions
While Apache Spark has a plethora of functions, the flexibility of Spark allows for data engineers and data scientists to define their own functions (i.e., user-defined functions or UDFs).

In [7]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my_app") .config("spark.memory.offHeap.enabled","true") .config("spark.memory.offHeap.size","10g").enableHiveSupport().getOrCreate()

In [ ]:
from pyspark.sql.types import LongType

# Create cubed function
def cubed(s):
    return s * s * s

# Register UDF
spark.udf.register("cubed", cubed, LongType())

# Generate temporary view
spark.range(1, 9).createOrReplaceTempView("udf_test")

In [ ]:
spark.udf.register("cubed", cubed, LongType())

In [ ]:
spark.range(1, 9).createOrReplaceTempView("udf_test")

In [ ]:
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test").show()

# Evaluation order and null checking in Spark SQL
Spark SQL (this includes SQL, the DataFrame API, and the Dataset API) does not
guarantee the order of evaluation of subexpressions. For example, the following query
does not guarantee that the s is NOT NULL clause is executed prior to the strlen(s) > 1 clause:
> spark.sql("SELECT s FROM test1 WHERE s IS NOT NULL AND strlen(s) > 1")


Therefore, to perform proper null checking, it is recommended that you do the
following:
1. Make the UDF itself null-aware and do null checking inside the UDF.
2. Use IF or CASE WHEN expressions to do the null check and invoke the UDF in a
conditional branch.

In [7]:
import pandas as pd
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the cubed function 
def cubed(a: pd.Series) -> pd.Series:
    return a * a * a

# Create the pandas UDF for the cubed function 
cubed_udf = pandas_udf(cubed, returnType=LongType())

In [8]:
#Using pandas dataframe
# Create a Pandas series
x = pd.Series([1, 2, 3])

# The function for a pandas_udf executed with local Pandas data
print(cubed(x))

0     1
1     8
2    27
dtype: int64


In [9]:
# Create a Spark DataFrame
df = spark.range(1, 4)

# Execute function as a Spark vectorized UDF
df.select("id", cubed_udf(col("id"))).show()

+---+---------+
| id|cubed(id)|
+---+---------+
|  1|        1|
|  2|        8|
|  3|       27|
+---+---------+



# Higher Order Functions in DataFrames and Spark SQL
Because complex data types are an amalgamation of simple data types, it is tempting to manipulate complex data types directly. As noted in the post Introducing New Built-in and Higher-Order Functions for Complex Data Types in Apache Spark 2.4 there are typically two solutions for the manipulation of complex data types.

Exploding the nested structure into individual rows, applying some function, and then re-creating the nested structure as noted in the code snippet below (see Option 1)
Building a User Defined Function (UDF) 

In [11]:
# Create an array dataset
arrayData = [[1, (1, 2, 3)], [2, (2, 3, 4)], [3, (3, 4, 5)]]

# Create schema
from pyspark.sql.types import *
#created the schema
arraySchema = (StructType([
      StructField("id", IntegerType(), True), 
      StructField("values", ArrayType(IntegerType()), True)
      ]))

# Create DataFrame on the basis of schema
df = spark.createDataFrame(spark.sparkContext.parallelize(arrayData), arraySchema)
df.createOrReplaceTempView("table")
df.printSchema()
df.show()

root
 |-- id: integer (nullable = true)
 |-- values: array (nullable = true)
 |    |-- element: integer (containsNull = true)

+---+---------+
| id|   values|
+---+---------+
|  1|[1, 2, 3]|
|  2|[2, 3, 4]|
|  3|[3, 4, 5]|
+---+---------+



In [16]:
#In this nested SQL statement, we first explode(values) which creates a new row (with the id) for each element (value) within values
spark.sql('SELECT id, explode(values) AS value FROM table').show()

+---+-----+
| id|value|
+---+-----+
|  1|    1|
|  1|    2|
|  1|    3|
|  2|    2|
|  2|    3|
|  2|    4|
|  3|    3|
|  3|    4|
|  3|    5|
+---+-----+



In [18]:
spark.sql("""
SELECT id, collect_list(value + 1) AS newValues
  FROM  (SELECT id, explode(values) AS value
        FROM table) x
 GROUP BY id
""").show()

+---+---------+
| id|newValues|
+---+---------+
|  1|[2, 3, 4]|
|  2|[3, 4, 5]|
|  3|[4, 5, 6]|
+---+---------+



# Option 2: User Defined Function
To perform the same task (adding a value of 1 to each element in values), we can also create a user defined function (UDF) that uses map to iterate through each element (value) to perform the addition operation. 

In [19]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import ArrayType

# Create UDF
def addOne(values):
    return [value + 1 for value in values]

# Register UDF
spark.udf.register("plusOneIntPy", addOne, ArrayType(IntegerType()))  

# Query data
spark.sql("SELECT id, plusOneIntPy(values) AS values FROM table").show()
# see its way more easier than above snippit 

+---+---------+
| id|   values|
+---+---------+
|  1|[2, 3, 4]|
|  2|[3, 4, 5]|
|  3|[4, 5, 6]|
+---+---------+



# Higher-Order Functions
In addition to the previously noted built-in functions, there are high-order functions that take anonymous lambda functions as arguments.

In [20]:
from pyspark.sql.types import *
schema = StructType([StructField("celsius", ArrayType(IntegerType()))])

t_list = [[35, 36, 32, 30, 40, 42, 38]], [[31, 32, 34, 55, 56]]
t_c = spark.createDataFrame(t_list, schema)
t_c.createOrReplaceTempView("tC")

# Show the DataFrame
t_c.show()

+--------------------+
|             celsius|
+--------------------+
|[35, 36, 32, 30, ...|
|[31, 32, 34, 55, 56]|
+--------------------+



# Transform
transform(array<T>, function<T, U>): array<U>

The transform function produces an array by applying a function to each element of an input array (similar to a map function)

In [21]:
# Calculate Fahrenheit from Celsius for an array of temperatures
spark.sql(
    """SELECT celsius, transform(celsius, t -> ((t * 9) div 5) + 32) as fahrenheit FROM tC""").show()

+--------------------+--------------------+
|             celsius|          fahrenheit|
+--------------------+--------------------+
|[35, 36, 32, 30, ...|[95, 96, 89, 86, ...|
|[31, 32, 34, 55, 56]|[87, 89, 93, 131,...|
+--------------------+--------------------+



In [23]:
# Filter temperatures > 38C for array of temperatures
spark.sql("""SELECT celsius, filter(celsius, t -> t > 38) as high FROM tC""").show()

+--------------------+--------+
|             celsius|    high|
+--------------------+--------+
|[35, 36, 32, 30, ...|[40, 42]|
|[31, 32, 34, 55, 56]|[55, 56]|
+--------------------+--------+



In [24]:
# Is there a temperature of 38C in the array of temperatures
spark.sql("""
SELECT celsius, exists(celsius, t -> t = 38) as threshold
FROM tC
""").show()

+--------------------+---------+
|             celsius|threshold|
+--------------------+---------+
|[35, 36, 32, 30, ...|     true|
|[31, 32, 34, 55, 56]|    false|
+--------------------+---------+



# DataFrames and Spark SQL Common Relational Operators
The power of Spark SQL is that it contains many DataFrame Operations (also known as Untyped Dataset Operations).

In the next section, we will focus on the following common relational operators:

Unions and Joins, 
Windowing, 
Modifications 

In [2]:
from pyspark.sql.functions import expr

# Set File Paths
delays_path = r"C:\Users\syed3\Downloads\LearningSparkV2-master\LearningSparkV2-master\databricks-datasets\learning-spark-v2\flights\departuredelays.csv"
airports_path = r"C:\Users\syed3\Downloads\LearningSparkV2-master\LearningSparkV2-master\databricks-datasets\learning-spark-v2\flights\airport-codes-na.txt"

# Obtain airports dataset
airports = spark.read.options(header="true", inferSchema="true", sep="\t").csv(airports_path)
airports.createOrReplaceTempView("airports_na")

# Obtain departure Delays data
delays = spark.read.options(header="true").csv(delays_path)
delays = (delays
          .withColumn("delay", expr("CAST(delay as INT) as delay"))
          .withColumn("distance", expr("CAST(distance as INT) as distance")))

delays.createOrReplaceTempView("departureDelays")

# Create temporary small table
foo = delays.filter(expr(""" 
            origin == 'SEA' AND 
            destination == 'SFO' AND 
            date like '01010%' AND 
            delay > 0""")) 

foo.createOrReplaceTempView("foo")

In [3]:
spark.sql("SELECT * FROM airports_na LIMIT 10").show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+



In [4]:
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+



In [5]:
spark.sql("SELECT * FROM foo LIMIT 10").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [3]:
# Union two tables
bar = delays.union(foo)
bar.createOrReplaceTempView("bars")
bar.filter(expr("origin == 'SEA' AND destination == 'SFO' AND date LIKE '01010%' AND delay > 0")).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [7]:
spark.sql("""
SELECT * 
FROM bars
WHERE origin = 'SEA' 
   AND destination = 'SFO' 
   AND date LIKE '01010%' 
   AND delay > 0
""").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [4]:
# Join Departure Delays data (foo) with flight info
foo.join(
  airports, 
  airports.IATA == foo.origin
).select("*").show()

+--------+-----+--------+------+-----------+-------+-----+-------+----+
|    date|delay|distance|origin|destination|   City|State|Country|IATA|
+--------+-----+--------+------+-----------+-------+-----+-------+----+
|01010710|   31|     590|   SEA|        SFO|Seattle|   WA|    USA| SEA|
|01010955|  104|     590|   SEA|        SFO|Seattle|   WA|    USA| SEA|
|01010730|    5|     590|   SEA|        SFO|Seattle|   WA|    USA| SEA|
+--------+-----+--------+------+-----------+-------+-----+-------+----+



In [6]:
spark.sql('DROP TABLE IF EXISTS departureDelaysWindow')
spark.sql("""
CREATE TABLE departureDelaysWindow AS
SELECT origin, destination, sum(delay) as TotalDelays 
  FROM departureDelays 
 WHERE origin IN ('SEA', 'SFO', 'JFK') 
   AND destination IN ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL') 
 GROUP BY origin, destination
""")
spark.sql("""SELECT * FROM departureDelaysWindow""").show()

# Modifications
Another common DataFrame operation is to perform modifications to the DataFrame. Recall that the underlying RDDs are immutable (i.e. they do not change) to ensure there is data lineage for Spark operations. Hence while DataFrames themselves are immutable, you can modify them through operations that create a new, different DataFrame with different columns, for example.

In [10]:
foo2 = foo.withColumn("status", expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END"))
foo2.show()
#The newly created foo2 DataFrame has the contents of the original foo DataFrame
#plus the additional status column defined by the CASE statement

+--------+-----+--------+------+-----------+-------+
|    date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
|01010710|   31|     590|   SEA|        SFO|Delayed|
|01010955|  104|     590|   SEA|        SFO|Delayed|
|01010730|    5|     590|   SEA|        SFO|On-time|
+--------+-----+--------+------+-----------+-------+



In [11]:
foo3 = foo2.drop("delay")
foo3.show()

+--------+--------+------+-----------+-------+
|    date|distance|origin|destination| status|
+--------+--------+------+-----------+-------+
|01010710|     590|   SEA|        SFO|Delayed|
|01010955|     590|   SEA|        SFO|Delayed|
|01010730|     590|   SEA|        SFO|On-time|
+--------+--------+------+-----------+-------+



In [12]:
#rename
foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()

+--------+--------+------+-----------+-------------+
|    date|distance|origin|destination|flight_status|
+--------+--------+------+-----------+-------------+
|01010710|     590|   SEA|        SFO|      Delayed|
|01010955|     590|   SEA|        SFO|      Delayed|
|01010730|     590|   SEA|        SFO|      On-time|
+--------+--------+------+-----------+-------------+



In [19]:
spark.sql('''select * from departureDelays''').show(10, False)

+--------+-----+--------+------+-----------+
|date    |delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|6    |602     |ABE   |ATL        |
|01020600|-8   |369     |ABE   |DTW        |
|01021245|-2   |602     |ABE   |ATL        |
|01020605|-4   |602     |ABE   |ATL        |
|01031245|-4   |602     |ABE   |ATL        |
|01030605|0    |602     |ABE   |ATL        |
|01041243|10   |602     |ABE   |ATL        |
|01040605|28   |602     |ABE   |ATL        |
|01051245|88   |602     |ABE   |ATL        |
|01050605|9    |602     |ABE   |ATL        |
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [17]:
#When working with your data, sometimes you will need to swap the columns for the rows—i.e., pivot your data.
spark.sql(
"""SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay 
FROM departureDelays WHERE origin = 'SEA'""").show(10)

+-----------+-----+-----+
|destination|month|delay|
+-----------+-----+-----+
|        ORD|    1|   92|
|        JFK|    1|   -7|
|        DFW|    1|   -5|
|        MIA|    1|   -3|
|        DFW|    1|   -3|
|        DFW|    1|    1|
|        ORD|    1|  -10|
|        DFW|    1|   -6|
|        DFW|    1|   -2|
|        ORD|    1|   -3|
+-----------+-----+-----+
only showing top 10 rows



Pivoting allows you to place names in the month column (instead of 1 and 2 you can
show Jan and Feb, respectively) as well as perform aggregate calculations (in this case
average and max) on the delays by destination and month

In [21]:
spark.sql("""
SELECT * FROM (
SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay 
  FROM departureDelays WHERE origin = 'SEA' 
) 
PIVOT (
  CAST(AVG(delay) AS DECIMAL(4, 2)) as AvgDelay, MAX(delay) as MaxDelay
  FOR month IN (1 JAN, 2 FEB, 3 MAR)
)
ORDER BY destination
""").show()

+-----------+------------+------------+------------+------------+------------+------------+
|destination|JAN_AvgDelay|JAN_MaxDelay|FEB_AvgDelay|FEB_MaxDelay|MAR_AvgDelay|MAR_MaxDelay|
+-----------+------------+------------+------------+------------+------------+------------+
|        ABQ|       19.86|         316|       11.42|          69|       11.47|          74|
|        ANC|        4.44|         149|        7.90|         141|        5.10|         187|
|        ATL|       11.98|         397|        7.73|         145|        6.53|         109|
|        AUS|        3.48|          50|       -0.21|          18|        4.03|          61|
|        BOS|        7.84|         110|       14.58|         152|        7.78|         119|
|        BUR|       -2.03|          56|       -1.89|          78|        2.01|         108|
|        CLE|       16.00|          27|        null|        null|        null|        null|
|        CLT|        2.53|          41|       12.96|         228|        5.16|  

In [14]:
spark.sql("""
SELECT * FROM (
SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay 
  FROM departureDelays WHERE origin = 'SEA' 
) 
PIVOT (
  CAST(AVG(delay) AS DECIMAL(4, 2)) as AvgDelay, MAX(delay) as MaxDelay
  FOR month IN (1 JAN, 2 FEB)
)
ORDER BY destination
""").show()

+-----------+------------+------------+------------+------------+
|destination|JAN_AvgDelay|JAN_MaxDelay|FEB_AvgDelay|FEB_MaxDelay|
+-----------+------------+------------+------------+------------+
|        ABQ|       19.86|         316|       11.42|          69|
|        ANC|        4.44|         149|        7.90|         141|
|        ATL|       11.98|         397|        7.73|         145|
|        AUS|        3.48|          50|       -0.21|          18|
|        BOS|        7.84|         110|       14.58|         152|
|        BUR|       -2.03|          56|       -1.89|          78|
|        CLE|       16.00|          27|        null|        null|
|        CLT|        2.53|          41|       12.96|         228|
|        COS|        5.32|          82|       12.18|         203|
|        CVG|       -0.50|           4|        null|        null|
|        DCA|       -1.15|          50|        0.07|          34|
|        DEN|       13.13|         425|       12.95|         625|
|        D

In [22]:
# check how Spark manages memory might help you understand the underlying concept